In [707]:
import pyodbc
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import precision_score
import pickle
from datetime import datetime
import warnings
conn = pyodbc.connect(
    "DRIVER={SQL Server};SERVER=lenovo-desktop;DATABASE=Qihuo;UID=samtsql;PWD=F(W}q:TsyK,7^+>`P28e79s#Uc5n")


def getData(symbol):
    query = f"""select top 2059 m1.date as 'Date',DATEPART(hour, m1.Date) AS 'Hour', m1.[Close] as 'Market', m2.[close] as 'Stock'  from MinuteQuote m1
    inner join MinuteQuote m2 on m1.Date = m2.date
    where m1.Date > getdate() - 45 and m1.Contract = 'ym' and m2.Contract = '{symbol}'
    order by m1.date desc"""

    with warnings.catch_warnings():
        warnings.simplefilter('ignore', UserWarning)
        df = pd.read_sql(query, conn)
        df = df.iloc[::-1]
        df.set_index('Date', inplace=True)
        return df


In [708]:
from enum import Enum
class ToTrade(Enum):
    OpenLong = 1
    OpenShort = 2
    Close = 3
    CloseLong = 4
    CloseShort = 5
    

In [709]:
def getCurrentPosition(symbol):
    query = f"""select * from ArbitrageMLTrade
                where AccountName = 'xieie181' and Symbol1 = '{symbol}' and Active = 1"""

    with warnings.catch_warnings():
        warnings.simplefilter('ignore', UserWarning)
        df = pd.read_sql(query, conn)
        if df.empty:
            return 0
        else:
            return df['Position'][0]


In [710]:
def OpenTrade(symbol, marketPrice, stockPrice, position):
    share2 = 1.0
    share1 = round((marketPrice * 5 / stockPrice) / 100, 0) * 100
    
    query = f"""INSERT INTO [dbo].[ArbitrageMLTrade]
           ([AccountName]
           ,[SecurityType1]
           ,[SecurityType2]
           ,[Symbol1]
           ,[Symbol2]
           ,[Position]
           ,[Share1]
           ,[Share2]
           ,[EnterPrice1]
           ,[EnterPrice2]
           ,[ExitPrice1]
           ,[ExitPrice2]
           ,[EnterTime]
           ,[ExitTime]
           ,[Active]
           ,[CreatedOn]
           ,[ModifiedOn])
     VALUES
           ('xieie181'
           ,'Stock'
           ,'Future'
           ,'{symbol}'
           ,'YM'
           ,{position}
           ,{share1}
           ,{share2}
           ,{stockPrice}
           ,{marketPrice}
           ,null
           ,null
           ,getdate()
           ,null
           ,1
           ,getdate()
           ,getdate())"""

    with warnings.catch_warnings():
        warnings.simplefilter('ignore', UserWarning)
        cursor = conn.cursor()
        cursor.execute(query)
        conn.commit()


In [711]:
def CloseTrade(symbol):
    query = f"""UPDATE [dbo].[ArbitrageMLTrade]
           SET Active = 0, ExitTime = GETDATE(), ModifiedOn = GETDATE() WHERE Symbol1 = '{symbol}' AND Active = 1"""

    with warnings.catch_warnings():
        warnings.simplefilter('ignore', UserWarning)
        cursor = conn.cursor()
        cursor.execute(query)
        conn.commit()

In [712]:
# def transformData(df):
    # df["StockReturn"] = df.rolling(2).apply(
    #     lambda x: x.iloc[1] / x.iloc[0] - 1)["Stock"]
    # df["MarketReturn"] = df.rolling(2).apply(
    #     lambda x: x.iloc[1] / x.iloc[0] - 1)["Market"]
    # df["OutPerform"] = df["StockReturn"] - df["MarketReturn"]
    # df["Target"] = (df.apply(lambda x: x > 0)["OutPerform"]).astype(int)


In [713]:
def getPredictors(df):
    predictors = ['Hour']
    for i in range(12):
        df[f'MarketReturn{pow(2,i)*5}'] = df.rolling(pow(2, i) + 1).apply(
            lambda x: x.iloc[pow(2, i)] / x.iloc[0] - 1)["Market"]
        predictors.append(f'OutPerform{pow(2,i)*5}')
        df[f'StockReturn{pow(2,i)*5}'] = df.rolling(pow(2, i) + 1).apply(
            lambda x: x.iloc[pow(2, i)] / x.iloc[0] - 1)["Stock"]
        predictors.append(f'StockReturn{pow(2,i)*5}')
        df[f"OutPerform{pow(2,i)*5}"] = df[f'StockReturn{pow(2,i)*5}'] - df[f'MarketReturn{pow(2,i)*5}']

    return predictors


In [714]:
def getMLdata(df, predictors):
    df = df[df.index.minute % 30 == 0]
    return df.iloc[-1]


In [715]:
def predict(test, predictors, model):
    preds = model.predict_proba(test[predictors])[:, 1]
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined


In [716]:
def processResult(currentPosition, predictions):
    prediction = predictions[len(predictions) - 1]

    if currentPosition == 0:
        if prediction > .57:
            return ToTrade.OpenLong
        elif prediction < .43:
            return ToTrade.OpenShort

    if currentPosition == 1:
        if prediction < .43:
            return ToTrade.CloseShort
        elif prediction < .50:
            return ToTrade.Close

    if currentPosition == -1:
        if prediction > .57:
            return ToTrade.CloseLong
        elif prediction > .50:
            return ToTrade.Close


In [717]:
# symbols = [
#     'CVX',
#     'HON',
#     'CRM',
#     'UNH',
#     'CSCO',
#     'WMT',
#     'AXP',
#     'JPM',
#     'MCD',
#     'HD',
#     'AMGN',
#     'V',
#     'INTC',
#     'WBA',
#     'GS',
#     'JNJ',
#     'PG',
#     'AAPL',
#     'DIS',
#     'MMM',
#     'MRK',
#     'MSFT',
#     'TRV',
#     'VZ',
#     'IBM',
#     'CAT',
#     'NKE']
# for s in symbols:
df = getData('msft')
# transformData(df)
predictors = getPredictors(df)
data = getMLdata(df, predictors)
    # with open(s, 'rb') as f:
    #     model = pickle.load(f)
    #     predictions = model.predict_proba(data[predictors])[:, 1]
    #     print(predictions)
    #     sys.stdout.flush()
    #     currentPosition = getCurrentPosition(s)
    #     totrade = processResult(currentPosition, predictions)
    #     if totrade == ToTrade.OpenLong:
    #         OpenTrade(s, df['Market'][len(df) - 1], df['Stock'][len(df) - 1], 1)
    #     elif totrade == ToTrade.OpenShort:     
    #         OpenTrade(s, df['Market'][len(df) - 1], df['Stock'][len(df) - 1], -1)
    #     elif totrade == ToTrade.Close:
    #         CloseTrade(s)   
    #     elif totrade == ToTrade.CloseLong:
    #         CloseTrade(s)
    #         OpenTrade(s, df['Market'][len(df) - 1], df['Stock'][len(df) - 1], 1)  
    #     elif totrade == ToTrade.CloseShort:  
    #         CloseTrade(s)
    #         OpenTrade(s, df['Market'][len(df) - 1], df['Stock'][len(df) - 1], -1)       

In [718]:
print(df)

                     Hour   Market   Stock  MarketReturn5  StockReturn5  \
Date                                                                      
2022-04-29 10:20:00    10  33342.0  284.03            NaN           NaN   
2022-04-29 10:25:00    10  33342.0  283.71       0.000000     -0.001127   
2022-04-29 10:30:00    10  33364.0  283.73       0.000660      0.000070   
2022-04-29 10:35:00    10  33383.0  283.48       0.000569     -0.000881   
2022-04-29 10:40:00    10  33322.0  282.92      -0.001827     -0.001975   
...                   ...      ...     ...            ...           ...   
2022-06-07 12:30:00    12  33133.0  272.43       0.000785      0.001139   
2022-06-07 12:35:00    12  33166.0  272.72       0.000996      0.001064   
2022-06-07 12:40:00    12  33168.0  272.41       0.000060     -0.001137   
2022-06-07 12:45:00    12  33173.0  272.54       0.000151      0.000477   
2022-06-07 12:50:00    12  33183.0  272.96       0.000301      0.001541   

                     Out

In [719]:
print(data)


Hour                    12.000000
Market               33133.000000
Stock                  272.430000
MarketReturn5            0.000785
StockReturn5             0.001139
OutPerform5              0.000354
MarketReturn10           0.001057
StockReturn10            0.001250
OutPerform10             0.000192
MarketReturn20           0.001966
StockReturn20            0.001544
OutPerform20            -0.000422
MarketReturn40           0.003149
StockReturn40            0.003019
OutPerform40            -0.000130
MarketReturn80           0.001390
StockReturn80            0.003019
OutPerform80             0.001629
MarketReturn160          0.005279
StockReturn160           0.005611
OutPerform160            0.000331
MarketReturn320          0.013490
StockReturn320           0.016757
OutPerform320            0.003268
MarketReturn640          0.004731
StockReturn640           0.008216
OutPerform640            0.003485
MarketReturn1280         0.003787
StockReturn1280          0.001802
OutPerform1280